In [25]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

In [26]:
# set parameters for CAPM. Choose your security, benchmark, risk free rate (or proxy for risk free rate), start & end dates for the downloaded data
RISKY_ASSET = 'AXP'
MARKET_BENCHMARK = '^SP500TR' #S&P 500 total return
MKT_RF ='^IRX' #90 T-bill
START_DATE = '2013-03-31'
END_DATE = '2023-03-31'

In [27]:
# create data frame of timeseries for asset, benchmark, and risk free rate proxy
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK, MKT_RF],start=START_DATE,end=END_DATE,progress=False)
# calculate returns
X = df['Adj Close'].rename(columns={RISKY_ASSET: 'asset', MARKET_BENCHMARK: 'market', MKT_RF: 'mkt_rf'}) \
 .resample('D') \
 .last() \
 .pct_change() \
 .dropna()

In [28]:
rf_series = X.loc[:,'mkt_rf'] # SHY return series 
X1 = X.subtract(rf_series, axis=0) # subtract RF return series from asset and benchmark series
X2= X1.drop(['mkt_rf'], axis=1) # Drop RF series (now all 0 values)

In [29]:
# covariance method for beta calculation (Cov(asset, benchmark) divided by volatility of market returns)
covariance = X2.cov().iloc[0,1]
benchmark_variance = X2.market.var()
beta = covariance / benchmark_variance
beta

1.0004984554746135

In [30]:
# CAPM regression setup
y = X2.pop('asset') #extract dependent variable (asset) times series
X2 = sm.add_constant(X2) # independent variable (benchmark) time series
capm_model = sm.OLS(y, X2).fit() # OLS regression from statsmodels
capm_model.summary() # Regression output. market coefficient should equal beta variable above from covariance method


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  asset   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 5.326e+06
Date:                Fri, 07 Apr 2023   Prob (F-statistic):               0.00
Time:                        17:10:16   Log-Likelihood:                 11182.
No. Observations:                3650   AIC:                        -2.236e+04
Df Residuals:                    3648   BIC:                        -2.235e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.408e-05      0.000      0.341      0.733      -0.000       0.000
market         1.0005      0.000   2307.762      0.000       1.000       1.001
==============================================================================
Omnibus:                     2300.997   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           408851.023
Skew:                           1.962   Prob(JB):                         0.00
Kurtosis:                      54.701   Cond. No.                         2.32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""